In [1]:
import os
import numpy as np
import zipfile
import io
import inspect
import cv2

from openvino.inference_engine import IECore
from openvino.runtime import Core

In [2]:
core = Core()

In [3]:
devices = core.available_devices

for device in devices:
    device_name = core.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

CPU:        Intel(R) Core(TM) i5-2410M CPU @ 2.30GHz


In [4]:
classification_model_xml = 'image-retrieval-0001/FP32/image-retrieval-0001.xml'

model = core.read_model(model=classification_model_xml)
compiled_model = core.compile_model(model=model, device_name="CPU")

In [5]:
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

In [6]:
print(f"input precision: {input_layer.element_type}")
print(f"input shape: {input_layer.shape}")

input precision: <Type: 'float32'>
input shape: [1,224,224,3]


In [7]:
def preprocess_image(image_path, h, w):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (w, h))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32)  # Convert to float32
#     image = image.transpose((2, 0, 1))  # Interchange dimensions to [1, 3, 224, 224]
    image = np.array(image)
    blob = np.expand_dims(image, axis=0)  # Add batch dimension
    print(blob.shape)
    return blob

In [8]:
query_image_path = 'images/knf.jpg'
preprocessed_query_image = preprocess_image(query_image_path, 224, 224)

(1, 224, 224, 3)


In [9]:
result_infer = compiled_model([preprocessed_query_image])[output_layer]
result_index = np.argmax(result_infer)

In [12]:
# Preprocess the batch of images
batch_images_path = 'archive/batchZip.zip'
batch_images_dir = 'unpack_archive'

with zipfile.ZipFile(batch_images_path, 'r') as zip_ref:
    zip_ref.extractall(batch_images_dir)
    zip_ref.close()

batch_images = []
for image_file in os.listdir(batch_images_dir):
    image_path = os.path.join(batch_images_dir, image_file)
    preprocessed_image = preprocess_image(image_path, 224, 224)
    batch_images.append(preprocessed_image)

batch_images = np.array(batch_images)

(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)


In [26]:
result_batch = []
for i in range(len(batch_images)):
    result = compiled_model([batch_images[i]])[output_layer]
    result_batch.append(result)

In [29]:
result_batch = np.array(result_batch)